# How to achieve Partial Immutability with Python's dataclass?
> Exploring the Python's `dataclass`

- toc: true 
- badges: true
- comments: true
- author: noklam
- hide: false
- categories: ["python"]

In [1]:
from dataclasses import dataclass, field, astuple

With `dataclass`, you can set `frozen=True` to ensure immutablilty.

In [2]:
@dataclass(frozen=True)
class FrozenDataClass:
    a: int
    b: int

frozen = FrozenDataClass(1,2)
frozen.c = 3

FrozenInstanceError: cannot assign to field 'c'

Mutating a frozen dataclass is not possible, but what if I need to compose some logic? `__post_init__()` method is how you can customize logic.

## __post_init__ assignment

In [3]:
@dataclass
class FrozenDataClass:
    a: int
    b: int
    
    def __post_init__(self):
        self.c = self.a + self.b
frozen = FrozenDataClass(1,2)

In [4]:
frozen.a, frozen.b, frozen.c

(1, 2, 3)

Do notice that I removed the `frozen=True` flag, see what happen if I put it back.

# What if you just want some of your attribute frozen?

## The good old `@property`?

In [9]:
@dataclass
class PartialFrozenDataClass:
    a: int # a should be frozen 
    b: int # Should be mutable
    
    @property
    def b(self):
        return self.b
    
p = PartialFrozenDataClass(1,2)

AttributeError: can't set attribute

It doesn't work!

## __post_init__ assignment in a frozen dataclass ✾

In [5]:
@dataclass(frozen=True)
class FrozenDataClass:
    a: int
    b: int
    
    def __post_init__(self):
        self.c = self.a + self.b
frozen = FrozenDataClass(1,2)

FrozenInstanceError: cannot assign to field 'c'

It doesn't work! Because the frozen flag will block any assignment even in the `__post_init__` method.

## workaround

In [10]:
@dataclass(frozen=True)
class FrozenDataClass:
    a: int
    b: int
    
    def __post_init__(self):
        object.__setattr__(self, 'c', self.a + self.b)
        
frozen = FrozenDataClass(1,2)
frozen.a, frozen.b, frozen.c

(1, 2, 3)

In [12]:
@dataclass(frozen=True)
class FrozenDataClass:
    a: int
    b: int
    
    def __post_init__(self):
        super().__setattr__('c', self.a + self.b)
        
frozen = FrozenDataClass(1,2)
frozen.a, frozen.b, frozen.c

(1, 2, 3)

In [19]:
frozen.c = 3

FrozenInstanceError: cannot assign to field 'c'

It works as expected, the workaround here is using `object.__setattr__`. The way `dataclass` achieve immutability is by blocking assignment in the `__setattr__` method. This trick works because we are using the `object` class method instead of the `cls` method, thus it won't stop us assign new attribute.  More details can be found in [Python Standard Doc](https://docs.python.org/3/library/dataclasses.html).

# Conclusion

You can't really use the normal `@property` trick either. The post-init assignment in a frozen dataclass is the only workaround. However, with `object.__setattr__` it confuses the IDE and it doesn't understand it is actually an member of the class which is kind of annoying.